In [3]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.{LongType, StringType, StructField, StructType}
import org.apache.spark.graphx._
import org.apache.spark.graphx.{Edge, Graph, PartitionID, VertexId}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{Column, DataFrame, Dataset, Row}
// import spark.implicits._

In [29]:
val record_schema = StructType(Array(
    StructField("ip", StringType, nullable=true),
    StructField("mac", StringType, nullable=true),
    StructField("hostname", StringType, nullable=true),
    StructField("serial_no", StringType, nullable=true),
    StructField("owner", StringType, nullable=true),
    StructField("source_name", StringType, nullable=true),
    StructField("id", LongType, nullable=false)
))

record_schema = StructType(StructField(ip,StringType,true), StructField(mac,StringType,true), StructField(hostname,StringType,true), StructField(serial_no,StringType,true), StructField(owner,StringType,true), StructField(source_name,StringType,true), StructField(id,LongType,false))


StructType(StructField(ip,StringType,true), StructField(mac,StringType,true), StructField(hostname,StringType,true), StructField(serial_no,StringType,true), StructField(owner,StringType,true), StructField(source_name,StringType,true), StructField(id,LongType,false))

In [30]:
val RecordId = "id"
def buildVerticesRDD(df: DataFrame): RDD[(VertexId, PartitionID)] = {
    def asVertexIdTuple(record: Row): (VertexId, PartitionID) = (record.getAs[VertexId](RecordId), 1)

    df.map(asVertexIdTuple).rdd
  }

RecordId = id


buildVerticesRDD: (df: org.apache.spark.sql.DataFrame)org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, org.apache.spark.graphx.PartitionID)]


id

In [31]:
val records = spark.read.format("csv")
                .option("header", "true")
                .schema(record_schema)
                .load("data/small_test_data.csv")
val mirrorColNames = for (col <- records.columns) yield "_"+col.toString
// val mirrorColNames = mirror.columns.foreach(col => "mirror_"+col.toString)
val mirror = records.toDF(mirrorColNames: _*)
// .withColumnRenamed("record_id", "mirror_record_id")
// 

records = [ip: string, mac: string ... 5 more fields]
mirrorColNames = Array(_ip, _mac, _hostname, _serial_no, _owner, _source_name, _id)
mirror = [_ip: string, _mac: string ... 5 more fields]


[_ip: string, _mac: string ... 5 more fields]

In [71]:
// records.printSchema
mirror.show()
// mirror.schema.foreach(e => println(e.name))
// records.schema.foreach(e => println(e.name))

+--------------+-----------------+--------------------+----------+----------------+------------+---+
|           _ip|             _mac|           _hostname|_serial_no|          _owner|_source_name|_id|
+--------------+-----------------+--------------------+----------+----------------+------------+---+
|          null|29:0b:58:3d:26:6d|isoulsby3v@behanc...|KXHGN8D8KM|    Izzy Soulsby|          LD|  0|
|          null|a9:11:e3:35:23:fa| cpollak74@nifty.com|Y5K30C65M3| Cathlene Pollak|          LD|  1|
|          null|90:22:03:25:11:d8|  ckissock53@php.net|7ZGA85QA5H|   Colin Kissock|          LD|  2|
|          null|4c:31:07:e9:9a:90|  bdruhan1m@1688.com|U0UOS9R43J|Brittaney Druhan|          LD|  3|
|106.60.189.138|             null|  bdruhan1m@1688.com|      null|            null|          SY|  4|
|106.60.189.138|4c:31:07:e9:9a:90|                null|      null|            null|          DT|  5|
+--------------+-----------------+--------------------+----------+----------------+--------

In [75]:
def cond(matchCols: Seq[String]): Column = {
    col("id")=!=col("_id") && matchCols.map(c => col(c)===col("_"+c)).reduce(_ || _)
//     
//     col("id")=!=col("_id") && col(matchCol).equalTo(col("_"+matchCol))
}

println(cond(Seq("ip", "mac")))
// println(cond("ip"))

val gggg = records.join(mirror, cond(Seq("ip", "mac")))
gggg.select("id", "_id").collect.foreach(println)

((NOT (id = _id)) AND ((ip = _ip) OR (mac = _mac)))
[3,5]
[4,5]
[5,3]
[5,4]


gggg = [ip: string, mac: string ... 12 more fields]


cond: (matchCols: Seq[String])org.apache.spark.sql.Column


[ip: string, mac: string ... 12 more fields]

In [73]:
val edges = records.join(mirror, records("id").notEqual(mirror("_id")) && 
                                 records("ip").equalTo(mirror("_ip")) ||
                                 records("mac").equalTo(mirror("_mac"))
                        )
//                      df1("record_id") =!= df2("record_id") )
//                            && (df1("ip") <=> df2("ip"))) )
// select($"record_id")
edges.select("id", "_id").collect.foreach(println)

[106.60.189.138,null,bdruhan1m@1688.com,null,null,SY,4,106.60.189.138,4c:31:07:e9:9a:90,null,null,null,DT,5]
[106.60.189.138,4c:31:07:e9:9a:90,null,null,null,DT,5,106.60.189.138,null,bdruhan1m@1688.com,null,null,SY,4]


edges = [ip: string, mac: string ... 12 more fields]


lastException: Throwable = null


[ip: string, mac: string ... 12 more fields]

In [9]:
val edges = records.join(mirror).where($"record_id".notEqual($"mirror_record_id") && 
                                       ($"ip".equalTo($"mirror_ip") ||
                                        $"mac".equalTo($"mirror_mac") ||
                                        $"hostname".equalTo($"mirror_hostname")
                                       ) 
                                      )
edges.select("record_id","mirror_record_id").distinct.show()
// .collect.foreach(println)

+---------+----------------+
|record_id|mirror_record_id|
+---------+----------------+
|        3|               5|
|        3|               4|
|        4|               5|
|        4|               3|
|        5|               3|
|        5|               4|
+---------+----------------+



edges = [ip: string, mac: string ... 12 more fields]


[ip: string, mac: string ... 12 more fields]

In [14]:
val out = for (col <- Seq("ip", "mac")) yield col.toString
println(out.reduce())

List(ip, mac)


out = List(ip, mac)


List(ip, mac)

In [7]:
val edges = records.join(mirror, ($"record_id"=!=$"mirror_record_id" && 
                                  $"ip"===$"mirror_ip")
                        )
val temp = edges.select("record_id","mirror_record_id").distinct
// .show()

// val asEdge: Row => Edge[PartitionID] = row => Edge[PartitionID](row.getAs[VertexId](0), row.getAs[VertexId](1), attr = 1)
// val edgesRDD = edges.map(asEdge).rdd

// val edgesDF = temp.map(edge => Edge[PartitionID](edge.getAs[VertexId](0), edge.getAs[VertexId](1), 1)).rdd
val edgesRDD = temp.map(edge => Edge(edge.getAs[VertexId](0), edge.getAs[VertexId](1))).rdd

edges = [ip: string, mac: string ... 12 more fields]
temp = [record_id: bigint, mirror_record_id: bigint]
edgesRDD = MapPartitionsRDD[16] at rdd at <console>:60


MapPartitionsRDD[16] at rdd at <console>:60

In [27]:
edgesRDD.collect()

Array(Edge(4,5,null), Edge(5,4,null))

In [112]:
val edgeRDD = edges.select("record_id","mirror_record_id").withColumn("edge_id", monotonically_increasing_id).rdd

edgeRDD = MapPartitionsRDD[320] at rdd at <console>:49


MapPartitionsRDD[320] at rdd at <console>:49

In [ ]:
// val vertices: RDD[(VertexId, _)] =
// def asVertexIdTuple(record: Row): (VertexId, PartitionID) = (record.getAs[VertexId]("record_id"), 1)
records.withColumn("id", monotonically_increasing_id).select("id").map(r => r.getAs[Int](0)).rdd.collect()
// .rdd.collect
// .map(_.record_id)

In [254]:
// val nodes: RDD[(VertexId, (String, String))] = 
records
// .map( r => r(0) )
// .collect.foreach(println)

[ip: string, mac: string ... 5 more fields]

In [31]:
val nodes = buildVerticesRDD(records)

nodes = MapPartitionsRDD[97] at rdd at <console>:38


MapPartitionsRDD[97] at rdd at <console>:38

In [29]:
nodes.collect()

Array((0,1), (1,1), (2,1), (3,1), (4,1), (5,1))

In [30]:
val nodes: RDD[(VertexId, Option[String])] = records.select("record_id").flatMap(array => array).map((_.toLong, None))

Name: Unknown Error
Message: <console>:46: error: type mismatch;
 found   : org.apache.spark.sql.Row
 required: TraversableOnce[?]
       val nodes: RDD[(VertexId, Option[String])] = records.select("record_id").flatMap(array => array).map((_.toLong, None))
                                                                                                 ^
<console>:46: error: missing parameter type for expanded function ((x$1) => x$1.toLong)
       val nodes: RDD[(VertexId, Option[String])] = records.select("record_id").flatMap(array => array).map((_.toLong, None))
                                                                                                             ^

StackTrace: 

In [34]:
val graph = Graph(nodes, edgesRDD)

Name: Unknown Error
Message: <console>:48: error: type mismatch;
 found   : org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Nothing]]
 required: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[ED]]
Error occurred in an application involving default arguments.
       val graph = Graph(nodes, edgesRDD)
                                ^

StackTrace: 

In [189]:
val users: RDD[(VertexId, (String))] =
  sc.parallelize(Array((3L, ("rxin")), (7L, ("jgonzal")),
                       (5L, ("franklin")), (2L, ("istoica"))))
// Create an RDD for edges
val relationships: RDD[Edge[String]] =
  sc.parallelize(Array(Edge(3L, 7L, "match"),    Edge(5L, 3L, "match"),
                       Edge(2L, 5L, "match"), Edge(5L, 7L, "match")))
// Define a default user in case there are relationship with missing user
// val defaultUser = ("John Doe", "Missing")
// Build the initial Graph
val graph = Graph(users, relationships)

users = ParallelCollectionRDD[450] at parallelize at <console>:61
relationships = ParallelCollectionRDD[451] at parallelize at <console>:65
graph = org.apache.spark.graphx.impl.GraphImpl@503343a8


org.apache.spark.graphx.impl.GraphImpl@503343a8